In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

df = pd.read_csv('../diamonds_good.csv')
df

,carat,cut,color,clarity,depth,table,price,'x','y','z'
0,0.23,Ideal,E,SI2,61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335.0,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757.0,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757.0,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757.0,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757.0,6.15,6.12,3.74


carat представляет собой вес алмаза в каратах, где 1 карат равен 200 миллиграммам. Это наиболее важный фактор при определении цены алмаза, так как вес напрямую влияет на размер и редкость камня. Более крупные алмазы встречаются реже и имеют нелинейное увеличение стоимости с ростом веса

Фичи cut (огранка), color (цвет) и clarity (чистота) представляют собой три из Четырёх C (4Cs) оценки качества алмазов по системе GIA

сut определяет качество огранки алмаза, которое влияет на его способность отражать и преломлять свет. Категории идут от Fair (посредственная) до Ideal (идеальная), где лучшая огранка максимально усиливает оптические свойства алмаза

color описывает степень бесцветности алмаза по шкале от D (полностью бесцветный, самый ценный) до J (заметный желтоватый или коричневатый оттенок). Чем ближе к D, тем выше ценность, так как совершенно бесцветные алмазы встречаются реже

clarity указывает на отсутствие внутренних дефектов (включений) и внешних дефектов (царапин, сколов). Шкала начинается с I1 (включения видны невооруженным глазом) до IF (внутренне безупречный). Более чистые алмазы ценятся выше, так как включения могут влиять на прочность и внешний вид

Фичи depth и table представляют собой пропорции алмаза. depth это отношение глубины алмаза к его среднему диаметру в процентах, влияющее на способность камня отражать свет. table это ширина верхней плоской грани (площадки) относительно диаметра алмаза. Идеальные пропорции (глубина 59-63%, площадка 53-58% для круглой огранки)

Фичи x, y, z представляют собой физические размеры алмаза в миллиметрах по трём осям: длина, ширина и глубина соответственно. Эти измерения используются для расчета объема и проверки пропорций камня

price это таргет, представляющий собой цену алмаза в долларах. Цена определяется сложным взаимодействием всех вышеперечисленных факторов, причем вес карат обычно имеет наибольшее влияние, за которым следуют цвет, чистота и огранка. Получаем задачу регрессии


In [3]:
df.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price      float64
'x'        float64
'y'        float64
'z'        float64
dtype: object

Исправляем названия колонок с лишними кавычками, чтоб избежать ошибок в будущем

In [4]:
df.columns = df.columns.str.replace("'", "")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335.0,4.34,4.35,2.75


Проверяем пропущенные значения, тк они могут нарушить работу мл моделей

In [5]:
print("Пропущенные значения:")
print(df.isna().sum())
print(f"\nВсего пропусков: {df.isna().sum().sum()}")

Пропущенные значения:
carat      498
cut          0
color        0
clarity      0
depth      990
table        0
price        0
x            0
y            5
z            0
dtype: int64

Всего пропусков: 1493


Анализ аномальных значений в размерах

In [6]:
print("Строки с нулевыми значениями размеров:")
print(df[(df['x'] == 0) | (df['y'] == 0) | (df['z'] == 0)])

print("\nЭкстремальные значения в размерах (y > 30):")
print(df[df['y'] > 30])

Строки с нулевыми значениями размеров:
       carat        cut color clarity  depth  table    price     x     y    z
2207    1.00    Premium     G     SI2   59.1   59.0   3142.0  6.55  6.48  0.0
2314    1.01    Premium     H      I1   58.1   59.0   3167.0  6.66  6.60  0.0
4791    1.10    Premium     G     SI2   63.0   59.0   3696.0  6.50  6.47  0.0
5471    1.01    Premium     F     SI2   59.2   58.0   3837.0  6.50  6.47  0.0
10167   1.50       Good     G      I1   64.0   61.0   4731.0  7.15  7.04  0.0
11182   1.07      Ideal     F     SI2   61.6   56.0   4954.0  0.00  6.62  0.0
11963   1.00  Very Good     H     VS2   63.3   53.0   5139.0  0.00  0.00  0.0
13601   1.15      Ideal     G     VS2   59.2   56.0   5564.0  6.88  6.83  0.0
15951   1.14       Fair     G     VS1   57.5   67.0   6381.0  0.00  0.00  0.0
24394   2.18    Premium     H     SI2   59.4   61.0  12631.0  8.49  8.45  0.0
24520   1.56      Ideal     G     VS2   62.2   54.0  12800.0  0.00  0.00  0.0
26123   2.25    Premium  

Удаляем строки с нулевыми размерами

In [7]:
df_clean = df[(df['x'] > 0) & (df['y'] > 0) & (df['z'] > 0)].copy()
df_clean = df_clean[df_clean['y'] < 30].copy()

print(f"Размер после очистки: {df_clean.shape}")
print(f"Удалено строк: {len(df) - len(df_clean)}")

Размер после очистки: (53913, 10)
Удалено строк: 27


Заполняем пропущенные значения. Для carat используем медиану по cut и color. Для depth используем медиану по cut. Для y используем общую медиану

In [8]:
df_clean['y'] = df_clean['y'].fillna(df_clean['y'].median())
df_clean['carat'] = df_clean['carat'].fillna(
    df_clean.groupby(['cut', 'color'])['carat'].transform('median')
)
df_clean['depth'] = df_clean['depth'].fillna(
    df_clean.groupby('cut')['depth'].transform('median')
)

print("Пропуски после заполнения:")
print(df_clean.isna().sum().sum())

Пропуски после заполнения:
0


Создаем новые признаки

Добавим фичу объем (valume) как произведение размеров xyz, тк он д лучше описывает размер алмаза чем отдельные измерения. Больший объем при том же весе может указывать на менее плотный камень

Цена за карат price_za_carat нормализует цену по весу, позволяя сравнивать алмазы разного размера. Высокая цена за карат может указывать на лучшее качество (лучший цвет, чистота, огранка)

Процент глубины depth_percentage используется для оценки пропорций алмаза, он может указывать на качество огранки и влиять на способность алмаза отражать свет

Площадь поверхности surface_area Площадь поверхности может влиять на блеск алмаза. Большая площадь поверхности при том же объеме может означать лучшую огранку

In [9]:
df_clean['volume'] = df_clean['x'] * df_clean['y'] * df_clean['z']
print(f"   Средний объем {df_clean['volume'].mean():.2f} мм^3")
print(f"   Корреляция объема с ценой {df_clean['volume'].corr(df_clean['price']):.4f}")

df_clean['price_za_carat'] = df_clean['price'] / df_clean['carat']
print(f"   Средняя цена за карат ${df_clean['price_za_carat'].mean():.2f}")
print(f"   Корреляция price_za_carat с price {df_clean['price_za_carat'].corr(df_clean['price']):.4f}")

df_clean['depth_percentage'] = (df_clean['depth'] / ((df_clean['x'] + df_clean['y']) / 2)) * 100
print(f"   Средний процент глубины {df_clean['depth_percentage'].mean():.1f}%")
print(f"   Корреляция depth_percentage с price {df_clean['depth_percentage'].corr(df_clean['price']):.4f}")

df_clean['surface_area'] = 2 * (df_clean['x']*df_clean['y'] + df_clean['x']*df_clean['z'] + df_clean['y']*df_clean['z'])
print(f"   Средняя площадь поверхности: {df_clean['surface_area'].mean():.2f} мм²")
print(f"   Корреляция surface_area с price: {df_clean['surface_area'].corr(df_clean['price']):.4f}")

print(f"Общее количество признаков: {df_clean.shape[1]}")


   Средний объем 129.81 мм^3
   Корреляция объема с ценой 0.9227
   Средняя цена за карат $4024.49
   Корреляция price_za_carat с price 0.8953
   Средний процент глубины 1118.1%
   Корреляция depth_percentage с price -0.8152
   Средняя площадь поверхности: 152.43 мм²
   Корреляция surface_area с price: 0.9101
Общее количество признаков: 14


Производим кодирование (что делали в первой лабораторной) по фичам, от которых зависит возрастание цены

In [17]:
cut_order = {'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4}
color_order = {'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6}
clarity_order = {'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7}

df_clean['cut_encoded'] = df_clean['cut'].map(cut_order)
df_clean['color_encoded'] = df_clean['color'].map(color_order)
df_clean['clarity_encoded'] = df_clean['clarity'].map(clarity_order)